# Bloco 1 — Infraestrutura e requirements.txt

In [ ]:
import sys, platform, os, json
from pathlib import Path
print("Python executable:", sys.executable)
print("Python version:", sys.version)
print("Platform:", platform.platform())
try:
    import importlib.metadata as im
except Exception:
    import importlib_metadata as im
pkgs = [
    "numpy","pandas","scikit-learn","scipy","matplotlib","seaborn",
    "nbformat","scikit-learn-extra","pyclustering","kaggle"
]
lines = []
for p in pkgs:
    try:
        v = im.version(p)
        print(f"{p}=={v}")
        lines.append(f"{p}=={v}")
    except Exception:
        print(f"{p} não instalado")
pins = {
    "scikit-learn":">=1.2", "numpy":">=1.26", "pandas":">=2.2", "scipy":">=1.10",
    "matplotlib":">=3.6", "seaborn":">=0.12", "nbformat":">=5.7",
    "scikit-learn-extra":">=0.3.0", "pyclustering":">=0.10.1.2", "kaggle":">=1.5"
}
for p in pkgs:
    if p in pins and not any(l.startswith(p+"==") for l in lines):
        lines.append(f"{p}{pins[p]}")
Path(".").joinpath("requirements.txt").write_text("\n".join(lines))
print("\nrequirements.txt gerado.")

print("\nConteúdo de requirements.txt:")
with open("requirements.txt","r") as f:
    print(f.read())

In [ ]:
!pip install -r requirements.txt

# Bloco 2 — Carregamento robusto do dataset

In [ ]:
import pandas as pd
def load_country_dataset():
    data_dir = Path("data")
    data_dir.mkdir(exist_ok=True)
    candidate_names = ["Country-data.csv", "country-data.csv", "country_data.csv"]
    for name in candidate_names:
        p = data_dir / name
        if p.exists():
            df = pd.read_csv(p)
            print("Carregado local:", p)
            return df, p
    try:
        from kaggle import api
        print("Baixando do Kaggle...")
        api.dataset_download_files(
            "rohan0301/unsupervised-learning-on-country-data",
            path=str(data_dir), unzip=True
        )
        for name in candidate_names:
            p = data_dir / name
            if p.exists():
                df = pd.read_csv(p)
                print("Baixado via Kaggle:", p)
                return df, p
    except Exception as e:
        print("Falha Kaggle:", e)
        raise FileNotFoundError(
            "Arquivo 'Country-data.csv' não encontrado em data/ e a API do Kaggle falhou. "
            "Coloque o CSV original na pasta data/."
        )
df, df_path = load_country_dataset()
print("Shape:", df.shape)
print(df.head(3))
expected_cols = {'country','child_mort','exports','health','imports','income','inflation','life_expec','total_fer','gdpp'}
assert expected_cols.issubset(set(df.columns)), f"Colunas inesperadas. Esperado conter: {expected_cols}"

# Bloco 3 — EDA: contagem de países, boxplots e histogramas

In [ ]:
n_countries = df['country'].nunique()
print("Número de países distintos:", n_countries)
import matplotlib.pyplot as plt
import math
numeric_cols = ['child_mort','exports','health','imports','income','inflation','life_expec','total_fer','gdpp']
print("Colunas numéricas:", numeric_cols)
rows = math.ceil(len(numeric_cols)/3)
df[numeric_cols].plot(kind='box', subplots=True, layout=(rows,3), figsize=(14, 4*rows), sharex=False, sharey=False)
plt.suptitle('Boxplots das variáveis numéricas')
plt.tight_layout()
plt.show()
fig, axes = plt.subplots(rows, 3, figsize=(14, 4*rows))
axes = axes.flatten()
for i, c in enumerate(numeric_cols):
    df[c].hist(bins=20, ax=axes[i])
    axes[i].set_title(c)
for j in range(i+1, rows*3):
    fig.delaxes(axes[j])
plt.tight_layout()
plt.show()

# Bloco 4 — Pré-processamento: imputação, log e padronização

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
X = df[numeric_cols].copy()
imp = SimpleImputer(strategy='median')
X_imp = pd.DataFrame(imp.fit_transform(X), columns=numeric_cols)
skews = X_imp.skew().abs()
log_cols = [c for c in numeric_cols if skews[c] > 1.0 and X_imp[c].min() > 0]
for c in log_cols:
    X_imp[c] = np.log1p(X_imp[c])
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imp), columns=numeric_cols)
Path("data").mkdir(exist_ok=True)
X_imp.to_csv("data/imputed_country_data.csv", index=False)
X_scaled.to_csv("data/preprocessed_country_data.csv", index=False)
print("Pré-processamento concluído. Shape:", X_scaled.shape)
display(X_scaled.describe().T)

# Bloco 5 — K-Means (k=3), centróides e país representante

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
k = 3
km = KMeans(n_clusters=k, random_state=42, n_init=10)
labels = km.fit_predict(X_scaled)
centers = km.cluster_centers_
res = df.copy()
res["kmeans_cluster"] = labels
closest_idx = pairwise_distances_argmin(centers, X_scaled.values)
repr_countries = res.iloc[closest_idx]["country"].tolist()
print("Inércia (SSE):", km.inertia_)
print("Centróides (espaço escalado):")
display(pd.DataFrame(centers, columns=numeric_cols))
for i, (idx, name) in enumerate(zip(closest_idx, repr_countries)):
    print(f"Cluster {i} — país representante:", name)
print("\nMédias no espaço original:")
display(res.groupby("kmeans_cluster")[numeric_cols].mean().round(2))
print("\nMédias no espaço escalado (para interpretar sinais +/-):")
tmp = X_scaled.copy()
tmp["kmeans_cluster"] = labels
display(tmp.groupby("kmeans_cluster")[numeric_cols].mean().round(2))

# Bloco 6 — Visualização PCA 2D dos clusters K-Means

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=42)
p2 = pca.fit_transform(X_scaled)
plt.figure(figsize=(8,6))
plt.scatter(p2[:,0], p2[:,1], c=labels, cmap="tab10", s=40)
plt.title("PCA — K-Means (k=3)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# Bloco 7 — Clusterização hierárquica (dendrograma) e comparação com K-Means

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
Z = linkage(X_scaled, method="ward")
plt.figure(figsize=(14,6))
dendrogram(Z, labels=res["country"].values, leaf_rotation=90, leaf_font_size=6)
plt.title("Dendrograma (ward)")
plt.tight_layout()
plt.show()
res["hcluster"] = fcluster(Z, t=3, criterion="maxclust") - 1
print("Contingência KMeans × HClust")
display(pd.crosstab(res["kmeans_cluster"], res["hcluster"]))

# Bloco 8 — K-Medoids (com fallback manual)

In [ ]:
try:
    from sklearn_extra.cluster import KMedoids
    kmed = KMedoids(n_clusters=3, random_state=42)
    res["kmedoids"] = kmed.fit_predict(X_scaled)
    med_idx = kmed.medoid_indices_
    print("Medoids (índices):", med_idx)
    for i, idx in enumerate(med_idx):
        print(f"Cluster {i} medoid:", res.iloc[idx]["country"])
except Exception as e:
    print("sklearn-extra indisponível, computando medoids manualmente com base no KMeans.")
    med_labels = []
    med_idx_list = []
    for cl in range(3):
        idxs = np.where(res["kmeans_cluster"].values==cl)[0]
        sub = X_scaled.values[idxs]
        dist_sum = np.sum(np.linalg.norm(sub[:,None,:]-sub[None,:,:], axis=2), axis=1)
        med_pos = idxs[np.argmin(dist_sum)]
        med_idx_list.append(med_pos)
        med_labels.extend([cl]*len(idxs))
    res["kmedoids"] = res["kmeans_cluster"]
    print("Medoids (manual, por KMeans):", [res.iloc[i]["country"] for i in med_idx_list])

# Bloco 9 — DBSCAN com estimativa de eps por k-distância

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
k_for_eps = 4
nn = NearestNeighbors(n_neighbors=k_for_eps)
nn.fit(X_scaled)
dists, _ = nn.kneighbors(X_scaled)
k_dists = np.sort(dists[:, -1])
plt.figure(figsize=(8,3))
plt.plot(k_dists)
plt.title(f"k-dist (k={k_for_eps}) para escolha de eps")
plt.ylabel("distância")
plt.xlabel("amostras ordenadas")
plt.show()
eps = float(np.percentile(k_dists, 90))
print("eps sugerido (p90 k-dist):", eps)
db = DBSCAN(eps=eps, min_samples=k_for_eps)
db_labels = db.fit_predict(X_scaled)
res["dbscan_label"] = db_labels
print("Contagem por rótulo DBSCAN:")
print(pd.Series(db_labels).value_counts())

# Bloco 10 — Interpretação automática dos clusters (K-Means)

In [ ]:
g_scaled = X_scaled.copy()
g_scaled["kmeans_cluster"] = res["kmeans_cluster"].values
means_z = g_scaled.groupby("kmeans_cluster")[numeric_cols].mean()
def summarize_cluster(zrow, hi=0.5, lo=-0.5):
    high = [c for c,v in zrow.items() if v >= hi]
    low = [c for c,v in zrow.items() if v <= lo]
    return high, low
print("Resumo automático por cluster (z-médias):")
for cl, zrow in means_z.iterrows():
    hi, lo = summarize_cluster(zrow)
    rep = repr_countries[cl]
    print(f"\nCluster {cl} — representante: {rep}")
    print(" • Altas:", ", ".join(hi) if hi else "—")
    print(" • Baixas:", ", ".join(lo) if lo else "—")
labels_desc = []
for cl, zrow in means_z.iterrows():
    if zrow["life_expec"]>0.5 and zrow["income"]>0.5 and zrow["gdpp"]>0.5 and zrow["child_mort"]<-0.5:
        labels_desc.append("Alto desenvolvimento")
    elif zrow["life_expec"]<-0.5 and zrow["income"]<-0.5 and zrow["gdpp"]<-0.5 and zrow["child_mort"]>0.5:
        labels_desc.append("Baixo desenvolvimento")
    else:
        labels_desc.append("Intermediário")
print("\nRótulos descritivos sugeridos por cluster (ordem 0..2):", labels_desc)

# Bloco 11 — Salvar artefatos e checklist da rubrica

In [ ]:
Path("output").mkdir(exist_ok=True)
res.to_csv("output/cluster_results_with_labels.csv", index=False)
with open("output/rubric_map.json","w") as f:
    json.dump({
        "Part1_requirements_txt": os.path.exists("requirements.txt"),
        "Downloaded_dataset": str(df_path),
        "Count_countries": int(n_countries),
        "EDA_boxplots": True,
        "Preprocessing_impute_scale": True,
        "KMeans_k3": True,
        "Hierarchical": True,
        "Medoids": True,
        "DBSCAN_explained": True,
    }, f, indent=2)
print("Arquivos salvos em output/: cluster_results_with_labels.csv, rubric_map.json")

# Bloco 12 — Respostas teóricas

## Etapas do K-Means até a convergência

1. **Inicialização:** Escolher k centróides iniciais (exemplo: k-means++).
2. **Atribuição:** Associar cada ponto ao centróide mais próximo (usando distância euclidiana).
3. **Atualização:** Recalcular cada centróide como a média dos pontos atribuídos ao cluster.
4. **Critério de parada:** Repetir atribuição e atualização até que não haja mudança nas atribuições ou a variação total (SSE) não diminua significativamente, ou até atingir o número máximo de iterações.

---

## Algoritmo de K-Médias modificado para usar medóides (representantes reais)

1. **Inicialização:** Escolher k medóides (pontos reais do conjunto de dados).
2. **Atribuição:** Associar cada ponto ao medóide mais próximo.
3. **Atualização:** Para cada cluster, escolher como novo medóide o ponto do cluster que minimiza a soma das distâncias a todos os demais pontos do cluster (não a média).
4. **Critério de parada:** Parar quando os medóides não mudam ou a soma das distâncias intra-cluster não melhora.

> Obs.: Esse é o algoritmo K-Medoids (exemplo: PAM). Ele garante que o representante é sempre um ponto real do conjunto.

---

## Por que K-Means é sensível a outliers?

O centróide é calculado como a média aritmética dos pontos do cluster. Um ponto extremo (outlier) pode deslocar fortemente essa média, alterando a posição do centróide, as fronteiras dos clusters e até “puxando” pontos que não deveriam pertencer ao cluster.

---

## Por que DBSCAN é mais robusto a outliers?

DBSCAN define clusters por densidade: regiões com densidade acima de um limiar (eps, min_samples) formam clusters; pontos isolados ou raros, que não estão conectados por densidade, são rotulados como ruído (-1). Assim, outliers não afetam “centros” de clusters e apenas ficam fora de qualquer cluster.